In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline

from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRFRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
data=pd.read_csv('age_of_marriage_data.csv')
data.head()

,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


In [3]:
def check_shape(df):
    rows,columns= df.shape
    print('Total number of rows:',rows)
    print('totla nuber of Columns:',columns)

In [4]:
check_shape(data)

Total number of rows: 2567
totla nuber of Columns: 10


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2567 entries, 0 to 2566
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2567 non-null   int64  
 1   gender           2538 non-null   object 
 2   height           2449 non-null   object 
 3   religion         1932 non-null   object 
 4   caste            2425 non-null   object 
 5   mother_tongue    2403 non-null   object 
 6   profession       2237 non-null   object 
 7   location         2412 non-null   object 
 8   country          2551 non-null   object 
 9   age_of_marriage  2548 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 200.7+ KB


In [6]:
data.describe()

,id,age_of_marriage
count,2567.000000,2548.000000
mean,1284.000000,29.648352
std,741.173394,2.802414
min,1.000000,20.000000
25%,642.500000,28.000000
50%,1284.000000,30.000000
75%,1925.500000,32.000000
max,2567.000000,36.000000


In [ ]:
prof = ProfileReport(data)
prof.to_file(output_file='output.html')

In [ ]:
for i in data.columns:
    print(data[i].value_counts())
    print('-----------------------------------------')

In [ ]:
# here we can drop the useless columns
# id columns has single value it wont help in our prediction so we can remove it

data.drop('id',axis=1,inplace=True)

In [ ]:
for i in data.columns:
    print(data[i].value_counts())
    print('-----------------------------------------')

In [ ]:
data.isna().sum()/data.shape[0]*100 # checking the percentage of null values

In [ ]:
data.shape

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.shape

In [ ]:
data['profession'].unique()

In [ ]:
data['height'] # here height is in foot and inch so we have to convert it to cm or m

In [ ]:
data.loc[1,'height']

In [ ]:
data.loc[1,'height'].split('\'')

In [ ]:
data.loc[1,'height'].split('\'')[0]

In [ ]:
int(data.loc[1,'height'].split('\'')[0])*30.48

In [ ]:
int(data.loc[1,'height'].split('\'')[1].replace('"',""))*2.54

In [ ]:
def height_convert(x):
    return int(x.split('\'')[0])*30.48+int(x.split('\'')[1].replace('"',""))*2.54

In [ ]:
data['height_cms']= data['height'].apply(height_convert)

In [ ]:
data.head()

In [ ]:
data.drop('height',axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
# performing label encoding for other 

In [ ]:
le= LabelEncoder()

In [ ]:
data.loc[:,['gender','religion','caste','mother_tongue','profession','location','country']]=data.loc[:,['gender','religion','caste','mother_tongue','profession','location','country']].apply(le.fit_transform)

In [ ]:
data.head()

In [ ]:
X= data.drop('age_of_marriage',axis=1)
y= data['age_of_marriage']

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0)

models={'Linear regression': LinearRegression(),
       'Tree': DecisionTreeRegressor(),
       'Random_forest': RandomForestRegressor(n_estimators=80,max_depth=11),
       'Ada_boost': AdaBoostRegressor(),
       'Xgboost': XGBRegressor(),
       'Xgbr_boost': XGBRFRegressor()}

train_score={}
test_score={}

for name,model in models.items():
    model.fit(X_train,y_train)
    train_score[name]=model.score(X_train,y_train)
    test_score[name]=model.score(X_test,y_test)

In [ ]:
train_score

In [ ]:
test_score

In [ ]:
result= pd.DataFrame(train_score,index=['Train_score'])
result= result.T
result['Test_score']= test_score.values()
result['Difference']= result['Train_score']-result['Test_score']
result

In [ ]:
result.plot.bar();

In [ ]:
rf= RandomForestRegressor(n_estimators=80,max_depth=11).fit(X_train,y_train)

In [ ]:
y_pred= rf.predict(X_test)

In [ ]:
mse= mean_squared_error(y_test,y_pred)
mae= mean_absolute_error(y_test,y_pred)
r_squre= r2_score(y_test,y_pred)
print('mean_squared_error:',mse)
print('\n')
print('mean_absolute_error:',mae)
print('\n')
print('r_square:',r_squre)